In [1]:
#"train/launcher.py train/train.py /mnt/config/parameters.yaml train_loader.dataset.split=train eval_loader.dataset.split=val"

from omegaconf import DictConfig, ListConfig
from omegaconf import OmegaConf as om

yaml_path = "/Users/yu.gong/workspace/mosaic/llm-foundry/scripts/train/yamls/pretrain/mpt-125m.yaml"
args_list = ["train_loader.dataset.split=train", "eval_loader.dataset.split=val"]
om.clear_resolver('oc.env')

    # Load yaml and cli arguments.
with open(yaml_path) as f:
    yaml_cfg = om.load(f)
cli_cfg = om.from_cli(args_list)
cfg = om.merge(yaml_cfg, cli_cfg)
om.resolve(cfg)

print(cfg)


{'data_local': './my-copy-c4', 'data_remote': None, 'max_seq_len': 2048, 'global_seed': 17, 'run_name': None, 'model': {'name': 'mpt_causal_lm', 'init_device': 'meta', 'd_model': 768, 'n_heads': 12, 'n_layers': 12, 'expansion_ratio': 4, 'max_seq_len': 2048, 'vocab_size': 50368, 'attn_config': {'attn_impl': 'triton'}}, 'tokenizer': {'name': 'EleutherAI/gpt-neox-20b', 'kwargs': {'model_max_length': 2048}}, 'train_loader': {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'train', 'shuffle': True, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': True, 'num_workers': 8}, 'eval_loader': {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'val', 'shuffle': False, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': False, 'num_workers': 8}, 'scheduler': {'name': 'cosine_with_warmup', 't_warmup': '100ba', 'alpha_f': 0.1}, 'optimizer': {'name': 'decoupled_adamw', 'lr': 0.0006, 'betas': [0.9, 0.95], 'eps': 1e-08, 'weight_decay': 0.

In [2]:
from llmfoundry.utils.config_utils import  pop_config
from typing import Any, Dict, List, Optional, Union
import copy
import gc
import logging
import os
import sys
import time
import warnings
from typing import Any, Dict, List, Optional, Union

import torch
from composer import Trainer
from composer.core.callback import Callback
from composer.loggers import MosaicMLLogger, MLFlowLogger
from composer.loggers.mosaicml_logger import (MOSAICML_ACCESS_TOKEN_ENV_VAR,
                                              MOSAICML_PLATFORM_ENV_VAR)
from composer.metrics.nlp import InContextLearningMetric
from composer.profiler import (JSONTraceHandler, Profiler, TraceHandler,
                               cyclic_schedule)
from composer.utils import dist, get_device, reproducibility
from omegaconf import DictConfig, ListConfig
from omegaconf import OmegaConf as om
from rich.traceback import install

install()

from transformers import PreTrainedTokenizerBase

from llmfoundry import COMPOSER_MODEL_REGISTRY
from llmfoundry.callbacks import AsyncEval
from llmfoundry.data.dataloader import build_dataloader
from llmfoundry.registry import import_file
from llmfoundry.utils.builders import (add_metrics_to_eval_loaders,
                                       build_algorithm, build_callback,
                                       build_evaluators, build_logger,
                                       build_optimizer, build_scheduler,
                                       build_tokenizer)
from llmfoundry.utils.config_utils import (log_config, pop_config,
                                           process_init_device,
                                           update_batch_size_info)
import contextlib
import logging
import math
import warnings
from typing import Any, Dict, Literal, Mapping, Optional, Tuple, Union

from composer.utils import dist
from omegaconf import DictConfig, ListConfig
from omegaconf import OmegaConf as om


code_paths = pop_config(cfg,
                            'code_paths',
                            must_exist=False,
                            default_value=[],
                            convert=True)

max_split_size_mb: Optional[int] = cfg.pop('max_split_size_mb', None)
cuda_load_lazy: bool = cfg.pop('cuda_load_lazy', False)
dist_timeout: Union[int, float] = pop_config(cfg,
                                                 'dist_timeout',
                                                 must_exist=False,
                                                 default_value=600.0)

print(f"code_paths: {code_paths}, max_split_size_mb: {max_split_size_mb}, cuda_load_lazy: {cuda_load_lazy}, dist_timeout: {dist_timeout}")
# glbal_train_batch_size 256, device_train_microbatch_size 16

code_paths: [], max_split_size_mb: None, cuda_load_lazy: False, dist_timeout: 600.0


In [3]:
model_config: DictConfig = pop_config(cfg, 'model', must_exist=True)
tokenizer_config: Dict[str, Any] = pop_config(cfg,
                                                  'tokenizer',
                                                  must_exist=True,
                                                  convert=True)
# tokenizer_config: {'name': 'EleutherAI/gpt-neox-20b', 'kwargs': {'model_max_length': 2048}}
# model: {'name': 'mpt_causal_lm', 'init_device': 'meta', 'd_model': 768, 'n_heads': 12, 'n_layers': 12, 'expansion_ratio': 4, 'max_seq_len': 2048, 'vocab_size': 50368, 'attn_config': {'attn_impl': 'triton'}}
print(f"tokenizer_config: {tokenizer_config}")
print(f"model: {model_config}")
   

tokenizer_config: {'name': 'EleutherAI/gpt-neox-20b', 'kwargs': {'model_max_length': 2048}}
model: {'name': 'mpt_causal_lm', 'init_device': 'meta', 'd_model': 768, 'n_heads': 12, 'n_layers': 12, 'expansion_ratio': 4, 'max_seq_len': 2048, 'vocab_size': 50368, 'attn_config': {'attn_impl': 'triton'}}


In [4]:
# optimizer_config: {'name': 'decoupled_adamw', 'lr': 0.0006, 'betas': [0.9, 0.95], 'eps': 1e-08, 'weight_decay': 0.0}
optimizer_config: Dict[str, Any] = pop_config(cfg,
                                                  'optimizer',
                                                  must_exist=True,
                                                  convert=True)
# scheduler_config: {'name': 'cosine_with_warmup', 't_warmup': '100ba', 'alpha_f': 0.1}
scheduler_config: Dict[str, Any] = pop_config(cfg,
                                                  'scheduler',
                                                  must_exist=True,
                                                  convert=True)
# train_loader_config: {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'train', 'shuffle': True, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': True, 'num_workers': 8}
train_loader_config: DictConfig = pop_config(cfg,
                                                 'train_loader',
                                                 must_exist=True)

# fsdp_config: {'sharding_strategy': 'FULL_SHARD', 'mixed_precision': 'PURE', 'activation_checkpointing': False, 'activation_checkpointing_reentrant': False, 'activation_cpu_offload': False, 'limit_all_gathers': True}
fsdp_config: Optional[Dict[str, Any]] = pop_config(cfg,
                                                       'fsdp_config',
                                                       must_exist=False,
                                                       default_value=None,
                                                       convert=True)
# eval_loader_config: {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'val', 'shuffle': False, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': False, 'num_workers': 8}
eval_loader_config: Optional[Union[DictConfig, ListConfig]] = pop_config(
        cfg, 'eval_loader', must_exist=False, default_value=None)

# icl_tasks_config: None
icl_tasks_config: Optional[Union[ListConfig,
                                     str]] = pop_config(cfg,
                                                        'icl_tasks',
                                                        must_exist=False,
                                                        default_value=None)


print(f"optimizer_config: {optimizer_config}")
print(f"scheduler_config: {scheduler_config}")
print(f"train_loader_config: {train_loader_config}")
print(f"fsdp_config: {fsdp_config}")
print(f"eval_loader_config: {eval_loader_config}")
print(f"icl_tasks_config: {icl_tasks_config}")

optimizer_config: {'name': 'decoupled_adamw', 'lr': 0.0006, 'betas': [0.9, 0.95], 'eps': 1e-08, 'weight_decay': 0.0}
scheduler_config: {'name': 'cosine_with_warmup', 't_warmup': '100ba', 'alpha_f': 0.1}
train_loader_config: {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'train', 'shuffle': True, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': True, 'num_workers': 8}
fsdp_config: {'sharding_strategy': 'FULL_SHARD', 'mixed_precision': 'PURE', 'activation_checkpointing': False, 'activation_checkpointing_reentrant': False, 'activation_cpu_offload': False, 'limit_all_gathers': True}
eval_loader_config: {'name': 'text', 'dataset': {'local': './my-copy-c4', 'remote': None, 'split': 'val', 'shuffle': False, 'max_seq_len': 2048, 'shuffle_seed': 17}, 'drop_last': False, 'num_workers': 8}
icl_tasks_config: None


In [5]:
eval_gauntlet_config: Optional[Union[DictConfig,
                                         str]] = pop_config(cfg,
                                                            'eval_gauntlet',
                                                            must_exist=False,
                                                            default_value=None)
icl_subset_num_batches: Optional[int] = pop_config(cfg,
    
                                                       'icl_subset_num_batches',
                                                       must_exist=False,
                                                       default_value=None)
icl_seq_len: Optional[int] = pop_config(cfg,
                                            'icl_seq_len',
                                            must_exist=False,
                                            default_value=None)
# eval_gauntlet_config: None, icl_subset_num_batches: None, icl_seq_len: None
print(f"eval_gauntlet_config: {eval_gauntlet_config}, icl_subset_num_batches: {icl_subset_num_batches}, icl_seq_len: {icl_seq_len}")

# Optional logging, evaluation and callback configs
logger_configs: Optional[DictConfig] = pop_config(cfg,
                                                      'loggers',
                                                      must_exist=False,
                                                      default_value=None,
                                                      convert=True)
callback_configs: Optional[DictConfig] = pop_config(cfg,
                                                        'callbacks',
                                                        must_exist=False,
                                                        default_value=None,
                                                        convert=True)
algorithm_configs: Optional[DictConfig] = pop_config(cfg,
                                                         'algorithms',
                                                         must_exist=False,
                                                         default_value=None)
# loggers: None, callbacks: {'speed_monitor': {'window_size': 10}, 'lr_monitor': {}, 'memory_monitor': {}, 'runtime_estimator': {}}, algorithms: {'gradient_clipping': {'clipping_type': 'norm', 'clipping_threshold': 1.0}}
print(f"loggers: {logger_configs}, callbacks: {callback_configs}, algorithms: {algorithm_configs}")

# Mandatory hyperparameters for training
device_train_batch_size: int = pop_config(cfg,
                                              'device_train_batch_size',
                                              must_exist=True)
device_eval_batch_size: int = pop_config(cfg,
                                             'device_eval_batch_size',
                                             must_exist=True)
max_duration: Union[int, str] = pop_config(cfg,
                                               'max_duration',
                                               must_exist=True)
eval_interval: Union[int, str] = pop_config(cfg,
                                                'eval_interval',
                                                default_value=1,
                                                must_exist=False)
precision: str = pop_config(cfg, 'precision', must_exist=True)
max_seq_len: int = pop_config(cfg, 'max_seq_len', must_exist=True)

print(f"device_train_batch_size: {device_train_batch_size}, device_eval_batch_size: {device_eval_batch_size}, max_duration: {max_duration}, eval_interval: {eval_interval}, precision: {precision}, max_seq_len: {max_seq_len}")


eval_gauntlet_config: None, icl_subset_num_batches: None, icl_seq_len: None
loggers: None, callbacks: {'speed_monitor': {'window_size': 10}, 'lr_monitor': {}, 'memory_monitor': {}, 'runtime_estimator': {}}, algorithms: {'gradient_clipping': {'clipping_type': 'norm', 'clipping_threshold': 1.0}}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:35                                                                                   │
│                                                                                                  │
│   32 print(f"loggers: {logger_configs}, callbacks: {callback_configs}, algorithms: {algorithm    │
│   33                                                                                             │
│   34 # Mandatory hyperparameters for training                                                    │
│ ❱ 35 device_train_batch_size: int = pop_config(cfg,                                              │
│   36 │   │   │   │   │   │   │   │   │   │   │     'device_train_batch_size',                    │
│   37 │   │   │   │   │   │   │   │   │   │   │     must_exist=True)                              │
│   38 device_eval_batch_size: int = pop_config(cfg,                                               │
│                                                                                                  │
│ /Users/yu.gong/miniconda3/envs/mosaic2/lib/python3.10/site-packages/llmfoundry/utils/config_util │
│ s.py:42 in pop_config                                                                            │
│                                                                                                  │
│    39 │   elif value is not None:                                                                │
│    40 │   │   return value                                                                       │
│    41 │   elif must_exist:                                                                       │
│ ❱  42 │   │   raise NameError(                                                                   │
│    43 │   │   │   f'The {key} parameter is missing and must exist for execution. Please check    │
│    44 │   │   )                                                                                  │
│    45 │   else:                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: The device_train_batch_size parameter is missing and must exist for execution. Please check your yaml.